## Ejercicio 1
##### A)  Interpole los puntos:
p1 = (5.4, 3.2)
p2_i = (9.5, 0.7)
p3 = (12.3, -3.6)
De estos, el punto p2i
  debe ser seleccionable y movible. Cree un programa que interpole una parábola en tiempo real para los tres puntos.

In [ ]:
#